In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
from dataclasses import dataclass
from math import radians

import numpy as np
import matplotlib.pyplot as plt

from cw.context import time_it
from cw.simulation import Simulation, StatesBase, AB3Integrator, LastValueLogger, Logging

from topone.sim_post_processing import sim_post_processing
from topone.environment import Environment, Stage
from topone.pid_agent import PIDAgent

In [3]:
@dataclass
class States(StatesBase):
    t: float = 0
    command_engine_on: bool = False
    command_drop_stage: bool = False
    gii: np.ndarray = np.zeros(2)
    xii: np.ndarray = np.zeros(2)
    vii: np.ndarray = np.zeros(2)
    aii: np.ndarray = np.zeros(2)
    fii_thrust: np.ndarray = np.zeros(2)
    theta: float = 0.
    theta_dot: float = 0.
    mass: float = 0.
    mass_dot: float = 0.
    h: float = 0.
    engine_on: bool = False
    stage_state: int = 0
    stage_idx: int = 0
    gamma_i: float = 0.
    gamma_e: float = 0.
    latitude: float = 0.

    def get_y_dot(self):
        y = np.empty(6)
        y[:2] = self.vii
        y[2:4] = self.aii
        y[4] = self.theta_dot
        y[5] = self.mass_dot
        return y

    def get_y(self):
        y = np.empty(6)
        y[:2] = self.xii
        y[2:4] = self.vii
        y[4] = self.theta
        y[5] = self.mass
        return y

    def set_t_y(self, t, y):
        self.t = t
        self.xii = y[:2]
        self.vii = y[2:4]
        self.theta = y[4]
        self.mass = y[5]

In [4]:
simulation = Simulation(
    states_class=States,
    integrator=AB3Integrator(
        h=0.1,
        rk4=True,
        fd_max_order=1),
    modules=[
        Environment(
            surface_diameter=1737.4e3,
            mu=4.9048695e12,
            stages=(
                Stage(
                    dry_mass=1,
                    propellant_mass=0.1,
                    specific_impulse=100,
                    thrust=2*0.17),
            ),
            initial_altitude=1000,
            initial_theta_e=radians(90),
            initial_latitude=radians(-30),
        ),
        PIDAgent(1, 0, 0)
    ],
    logging=Logging(),
    initial_state_values=None,
)

simulation.initialize()

In [5]:
with time_it("simulation run"):
    result = simulation.run(10)
result

simulation run: 0.005768670998804737 [s]


<xarray.Dataset>
Dimensions:             (d_2_0: 2, t: 10)
Coordinates:
  * t                   (t) float64 0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9
Dimensions without coordinates: d_2_0
Data variables:
    command_engine_on   (t) bool False False False False ... False False False
    command_drop_stage  (t) bool False False False False ... False False False
    gii                 (t, d_2_0) float64 -1.406 0.8115 ... -1.406 0.8115
    xii                 (t, d_2_0) float64 1.505e+06 -8.692e+05 ... -8.692e+05
    vii                 (t, d_2_0) float64 0.0 0.0 -0.1406 ... -1.265 0.7304
    aii                 (t, d_2_0) float64 -1.406 0.8115 ... -1.406 0.8115
    fii_thrust          (t, d_2_0) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    theta               (t) float64 -0.5236 -0.5236 -0.5236 ... -0.5236 -0.5236
    theta_dot           (t) float64 1.11e-16 -1.11e-16 ... -1.11e-16 -2.22e-16
    mass                (t) float64 1.1 1.1 1.1 1.1 1.1 1.1 1.1 1.1 1.1 1.1
    mass_dot            (t) int64 0 0 0 0 0 0 0 0 0 0
    h                   (t) float64 1e+03 1e+03 1e+03 ... 999.6 999.5 999.3
    engine_on           (t) bool False False False False ... False False False
    stage_state         (t) int64 0 0 0 0 0 0 0 0 0 0
    stage_idx           (t) int64 0 0 0 0 0 0 0 0 0 0
    gamma_i             (t) float64 0.0 2.618 2.618 2.618 ... 2.618 2.618 2.618
    gamma_e             (t) float64 2.094 4.712 4.712 ... 4.712 4.712 4.712
    latitude            (t) float64 -0.5236 -0.5236 -0.5236 ... -0.5236 -0.5236
Attributes:
    env_surface_diameter:  1737400.0
    env_mu:                4904869500000.0
    env_stages:            (Stage(dry_mass=1, propellant_mass=0.1, specific_i...
    env_initial_latitude:  -0.5235987755982988
    env_initial_altitude:  1000
    initial_theta_e:       1.5707963267948966

In [6]:
xie = sim_post_processing(result)

In [7]:
result.theta

<xarray.DataArray 'theta' (t: 10)>
array([-0.52359878, -0.52359878, -0.52359878, -0.52359878, -0.52359878,
       -0.52359878, -0.52359878, -0.52359878, -0.52359878, -0.52359878])
Coordinates:
  * t        (t) float64 0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9

In [8]:
result.latitude

<xarray.DataArray 'latitude' (t: 10)>
array([-0.52359878, -0.52359878, -0.52359878, -0.52359878, -0.52359878,
       -0.52359878, -0.52359878, -0.52359878, -0.52359878, -0.52359878])
Coordinates:
  * t        (t) float64 0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9

In [147]:
plt.figure()
plt.plot(result.xii.sel(d_2_0=0), result.xii.sel(d_2_0=1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [148]:
plt.figure()
result.aii.plot.line(x="t")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
plt.figure()
result.h.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …